In [ ]:
from bastionlab import Identity

# Create `Identity` for Data owner.
data_owner = Identity.create("data_owner")

In [ ]:
!wget 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'

In [ ]:
!python datagen.py

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
!unzip smsspamcollection.zip

In [2]:
import polars as pl

file_path = "./SMSSpamCollection"

labels = []
texts = []
with open(file_path) as f:
    for line in f.readlines():
        split = line.split("\t")
        labels.append(1 if split[0] == "spam" else 0)
        texts.append(split[1])
df = pl.DataFrame({"label": labels, "text": texts})
from bastionlab import Connection

connection = Connection("localhost")

from bastionlab.polars.policy import Policy, Aggregation, Log

policy = Policy(safe_zone=Aggregation(min_agg_size=10), unsafe_handling=Log())
rdf = connection.client.polars.send_df(df.limit(500), policy=policy, sanitized_columns=["text"])
test_rdf = connection.client.polars.send_df(
    df.tail(int(df.height / 3)).limit(250), policy=policy, sanitized_columns=["text"]
)
import torch
from typing import List, Optional, Union, Tuple


train_rds = rdf.to_dataset(["text"], "label", "distilbert-base-uncased")
test_rds = test_rdf.to_dataset(["text"], "label", "distilbert-base-uncased")

train_rds._set_test_dataset(test_rds.train_dataset_ref)
test_rds._update(train_dataset=train_rds.train_dataset_ref)

identifier: "bcda335b-eb73-4fc2-b4e1-c460975c5438"
meta: "\n\005\n\003\372\001j\n\005\n\003\372\001j\022\005Int64\022\005Int64\030\372\001"



In [ ]:
! pip install transformers

In [3]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
from bastionlab.torch.utils import MultipleOutputWrapper

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False,
    torchscript=True,
)
model = MultipleOutputWrapper(model, 0)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classi

In [4]:
remote_datasets = connection.client.torch.list_remote_datasets()
print(remote_datasets)

print([r.trace_input for r in remote_datasets])

__list__ [ds] [(identifier: "bcda335b-eb73-4fc2-b4e1-c460975c5438"
meta: "\n\005\n\003\372\001j\n\005\n\003\372\001j\022\005Int64\022\005Int64\030\372\001*G\n$4974a0cf-c473-422b-8e51-027bbb5598b1\"\037\n\005\n\003\364\003a\n\005\n\003\364\003a\022\005Int64\022\005Int64\030\364\003"
, identifier: "4974a0cf-c473-422b-8e51-027bbb5598b1"
meta: "\n\005\n\003\364\003a\n\005\n\003\364\003a\022\005Int64\022\005Int64\030\364\003"
), (identifier: "01d962bb-9f69-4b5f-82f9-6857cea9f4a9"
meta: "\n\005\n\003\364\003a\n\005\n\003\364\003a\022\005Int64\022\005Int64\030\364\003"
, None), (identifier: "80f41419-30c8-4019-9dff-cbda62bfde1a"
meta: "\n\005\n\003\372\001j\n\005\n\003\372\001j\022\005Int64\022\005Int64\030\372\001*G\n$01d962bb-9f69-4b5f-82f9-6857cea9f4a9\"\037\n\005\n\003\364\003a\n\005\n\003\364\003a\022\005Int64\022\005Int64\030\364\003"
, identifier: "01d962bb-9f69-4b5f-82f9-6857cea9f4a9"
meta: "\n\005\n\003\364\003a\n\005\n\003\364\003a\022\005Int64\022\005Int64\030\364\003"
), (identifi

In [5]:
from bastionlab.torch.optimizer_config import Adam
remote_learner = connection.client.torch.RemoteLearner(
    model,
    remote_datasets[0],
    max_batch_size=2,
    loss="cross_entropy",
    optimizer=Adam(lr=5e-5),
    model_name="DistilBERT",
)

RuntimeError: The size of tensor a (106) must match the size of tensor b (250) at non-singleton dimension 2

In [ ]:
import polars as pl

df = pl.DataFrame(
    {
        "0": [1, 2, 3, 4, 5, 6]
    }
)

df.get_column("0").reshape([2,3])